# **Periodic LiH VQE energy evaluation on quantum hardware with optimized parameters from simulation.**
# 
This notebook contains several components including:

1. How to load a Hamiltonian based on the Broombridge schema into qiskit including the functions needed to transform from spatial orbital notation in Broombridge to the spin orbital notation used by quiskit.

2. How to compute optimized parameters for a VQE calculation based on a hardware efficient ansatz using a noise-free simulator.

3. How to simulate with noise and with or without error mitigation.

4. Using the optimized parameters from the noise-free simulator, carry out an energy evaluation on hardware with error mitigation.

In [ ]:
import os
import yaml
from yaml import SafeLoader

from qiskit.aqua.algorithms import VQE, NumPyEigensolver
import matplotlib.pyplot as plt
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.circuit.library import EfficientSU2
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit.aqua.operators import Z2Symmetries
from qiskit import IBMQ, BasicAer, Aer, QuantumCircuit, QuantumRegister, transpile, assemble, execute
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter, complete_meas_cal
from qiskit.providers.aer.noise import NoiseModel
from qiskit.result import marginal_counts

**Creating a yaml file based on the Broombridge schema. This example is of LiH with integrals derived from periodic plane-wave calculations.**

In [ ]:
%%writefile H1Li1-7.0.yaml

"$schema": https://raw.githubusercontent.com/Microsoft/Quantum/master/Chemistry/Schema/broombridge-0.1.schema.json

bibliography:
- url: https://www.nwchem-sw.org
format:
  version: '0.1'
generator:
  source: nwchem
  version: '6.8'
integral_sets:
- basis_set:
    name: PW
    type: gaussian
  coulomb_repulsion:
    units: hartree
    value: 0.0
  energy_offset:
    units: hartree
    value: 0.0
  fci_energy:
    lower: -0.1
    units: hartree
    upper: 0.1
    value: 0.0
  geometry:
    atoms:
    - coords:
      - 0.0
      - 0.0
      - -1.75
      name: Li
    - coords:
      - 0.0
      - 0.0
      - 5.25
      name: H
    coordinate_system: cartesian
    units: angstrom
  hamiltonian:
    one_electron_integrals:
      format: sparse
      units: hartree
      values:
      - - 1
        - 1
        - -0.3219447128
      - - 2
        - 1
        - 0.1137558934
      - - 2
        - 2
        - -0.145848565
    two_electron_integrals:
      format: sparse
      index_convention: mulliken
      units: hartree
      values:
      - - 1
        - 1
        - 1
        - 1
        - 0.1555278586
      - - 1
        - 1
        - 2
        - 1
        - -0.113748421
      - - 1
        - 1
        - 2
        - 2
        - -3.2818e-06
      - - 2
        - 1
        - 2
        - 1
        - 0.077403184
      - - 2
        - 1
        - 2
        - 2
        - -0.0119685279
      - - 2
        - 2
        - 2
        - 2
        - 0.0038563053
  metadata:
    molecule_name: unknown
  initial_state_suggestions:
  - state:
      energy: {units: hartree, value: -0.0}
      label: '|G>'
      superposition:
      - [1.0, (1a)+, (1b)+, '|0>']
  n_electrons: 2
  n_orbitals: 2
  scf_energy:
    units: hartree
    value: 0.0
  scf_energy_offset:
    units: hartree
    value: 0.0


**Extract the Hamiltonian elements and other information from the yaml file, convert it to the format used by Qiskit, and prepare the qubit operator representing the molecule's Hamiltonian.**

In [ ]:
def get_spatial_integrals(one_electron,two_electron,n_orb):
    one_electron_spatial_integrals = np.zeros((n_orb, n_orb))
    two_electron_spatial_integrals = np.zeros((n_orb, n_orb, n_orb, n_orb))

    for ind, val in enumerate(one_electron):
        # This is because python index starts at 0
        i = int(val[0] - 1)
        j = int(val[1] - 1)
        one_electron_spatial_integrals[i, j] = val[2]
        if i != j:
            one_electron_spatial_integrals[j, i] = val[2]

    for ind, val in enumerate(two_electron):
        i = int(val[0]-1)
        j = int(val[1]-1)
        k = int(val[2]-1)
        l = int(val[3]-1)
        two_electron_spatial_integrals[i, j, k, l] = val[4]
        if two_electron_spatial_integrals[k, l, i, j] == 0:
            two_electron_spatial_integrals[k, l, i, j] = val[4]
        if two_electron_spatial_integrals[i, j, l, k] == 0:
            two_electron_spatial_integrals[i, j, l, k] = val[4]
        if two_electron_spatial_integrals[l, k, i, j] == 0:
            two_electron_spatial_integrals[l, k, i, j] = val[4]
        if two_electron_spatial_integrals[j, i, k, l] == 0:
            two_electron_spatial_integrals[j, i, k, l] = val[4]
        if two_electron_spatial_integrals[k, l, j, i] == 0:
            two_electron_spatial_integrals[k, l, j, i] = val[4]
        if two_electron_spatial_integrals[j, i, l, k] == 0:
            two_electron_spatial_integrals[j, i, l, k] = val[4]
        if two_electron_spatial_integrals[l, k, j, i] == 0:
            two_electron_spatial_integrals[l, k, j, i] = val[4]

    return one_electron_spatial_integrals, two_electron_spatial_integrals

def convert_to_spin_index(one_electron, two_electron,n_orb):
    h1 = np.block([[one_electron, np.zeros((int(n_orb), int(n_orb)))],
                   [np.zeros((int(n_orb), int(n_orb))), one_electron]])
    h2 = np.zeros((2 * n_orb, 2 * n_orb, 2 * n_orb, 2 * n_orb))

    for i in range(len(two_electron)):
        for j in range(len(two_electron)):
            for k in range(len(two_electron)):
                for l in range(len(two_electron)):

                    h2[i,j, k + n_orb, l + n_orb] = two_electron[i, j, k, l]
                    h2[i + n_orb, j + n_orb,k, l] = two_electron[i, j, k, l]

                    if i!=k and j!=l:
                        h2[i,j,k,l] = two_electron[i,j,k,l]
                        h2[i + n_orb, j + n_orb, k + n_orb, l + n_orb] = two_electron[i, j, k, l]
    return h1, 0.5*h2



data = yaml.load(open("H1Li1-7.0.yaml","r"),SafeLoader)
n_electrons = data['integral_sets'][0]['n_electrons']
n_spatial_orbitals = data['integral_sets'][0]['n_orbitals']
nuclear_repulsion_energy = data['integral_sets'][0]['coulomb_repulsion']['value']

one_electron_import = data['integral_sets'][0]['hamiltonian']['one_electron_integrals']['values']
two_electron_import = data['integral_sets'][0]['hamiltonian']['two_electron_integrals']['values']

one_electron_spatial_integrals, two_electron_spatial_integrals = get_spatial_integrals(one_electron_import,two_electron_import,n_spatial_orbitals)

h1, h2 = convert_to_spin_index(one_electron_spatial_integrals,two_electron_spatial_integrals,n_spatial_orbitals)

qubitOp = FermionicOperator(h1, h2).mapping(map_type='parity')
qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, n_electrons)

In [ ]:
from qiskit.test.mock import FakeVigo
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

backend = AerSimulator()
device_backend = FakeVigo()
device = QasmSimulator.from_backend(device_backend)
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)

1. Compute the exact/FCI energy.
2. Configure the optimizer, the variational form, and the VQE instance. This example uses a heuristic variational form (RYRZ) rather than UCCSD as RYRZ has a much shallower circuit than UCCSD and uses fewer two qubit gates.
3. Optimized parameters are assigned to ```p0```.

In [ ]:
quantum_instance = QuantumInstance(backend=backend, shots=500)

exact_solution = NumPyEigensolver(qubitOp).run()
print("Exact Result:", np.real(exact_solution.eigenvalues) + nuclear_repulsion_energy)

optimizer = SPSA(maxiter=100)
var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear")
vqe = VQE(qubitOp, var_form, optimizer=optimizer)
ret = vqe.run(quantum_instance)
vqe_result = np.real(ret['eigenvalue'] + nuclear_repulsion_energy)
print("VQE Result:", vqe_result)

p0 = ret.optimal_point
print("Optimal Parameters = ", p0)

**Here we declare the provider and which backend we want to use.**

In [ ]:
from azure.quantum.qiskit import AzureQuantumProvider
provider = AzureQuantumProvider(
    subscription_id = "***********",
    resource_group = "*********",
    name = "*********",
    location = "*********"
)

quantinuum_backend = provider.get_backend("quantinuum.hqs-lt-s1-sim")

**Evaluate the energy with noise (Quantinuum Simulator) and not error mitigation.**

In [ ]:
quantum_instance = QuantumInstance(backend=quantinuum_backend,
                                   shots=500)

quantum_instance._qjob_config = {}

optimizer = SPSA(maxiter=100)
var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear")
vqe = VQE(qubitOp, var_form, optimizer=optimizer, initial_point=p0)

# Set the quantum instance manually
vqe.quantum_instance = quantum_instance

vqe._energy_evaluation(parameters=p0) + nuclear_repulsion_energy

**Evaluate the energy with noise (Quantinuum Simulator) and with error mitigation.**

In [ ]:
quantum_instance = QuantumInstance(backend=quantinuum_backend,
                                   shots=500,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=1000)

quantum_instance._qjob_config = {}

optimizer = SPSA(maxiter=100)
var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear")
vqe = VQE(qubitOp, var_form, optimizer=optimizer, initial_point=p0)

# Set the quantum instance manually
vqe.quantum_instance = quantum_instance

vqe._energy_evaluation(parameters=p0) + nuclear_repulsion_energy

**Choose the hardware you want to use.**

In [ ]:
quantinuum_harware = provider.get_backend("quantinuum.hqs-lt-s1")

**Execute on the quantum hardware.**

In [ ]:
quantum_instance = QuantumInstance(backend=quantinuum_harware,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=1000)

quantum_instance._qjob_config = {}

optimizer = SPSA(maxiter=1)
var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear")
vqe = VQE(qubitOp, var_form, optimizer=optimizer, initial_point=p0)

# Set the quantum instance manually
vqe.quantum_instance = quantum_instance

vqe._energy_evaluation(parameters=p0) + nuclear_repulsion_energy

**If the notebook times out, you can calculate the energy from the job IDs. Replace the job names and IDs with those from your calculation.**

In [ ]:
jobs = {'mcalcal_00': '0165c616-f19b-11ec-97c4-00155d6895f8',
 'mcalcal_01': '01d3ba68-f19b-11ec-97c4-00155d6895f8',
 'mcalcal_10': '022724e6-f19b-11ec-97c4-00155d6895f8',
 'mcalcal_11': '026b32f8-f19b-11ec-97c4-00155d6895f8',
 'EfficientSU2-988-2105': '02b25700-f19b-11ec-97c4-00155d6895f8',
 'EfficientSU2-988-2106': '02fb668e-f19b-11ec-97c4-00155d6895f8',
 'EfficientSU2-988-2107': '03411080-f19b-11ec-97c4-00155d6895f8',
 'EfficientSU2-988-2108': '0390d048-f19b-11ec-97c4-00155d6895f8'}

jobs["EfficientSU2"] = [
    job for name, job in sorted(jobs.items(), reverse=True) if "EfficientSU2" in name
]

def fetch_job(qobj, **kwargs):
    name = qobj.experiments[0].header.name
    if name.startswith("EfficientSU2"):
        job_id = jobs["EfficientSU2"].pop()
    else:
        job_id = jobs.get(name)
    if job_id is not None:
        job = quantinuum_backend.retrieve_job(job_id=job_id)
        job.refresh()
        return job
    else:
        raise ValueError(f"No job found with name '{name}'.")

quantinuum_backend.run = fetch_job

quantum_instance = QuantumInstance(backend=quantinuum_backend,
                                   measurement_error_mitigation_cls=CompleteMeasFitter,
                                   cals_matrix_refresh_period=1000)

# Unset qjob config to avoid errors when running job.result()
quantum_instance._qjob_config = {}
# Create optimizer with only one iteration
optimizer = SPSA(maxiter=1)
var_form = EfficientSU2(qubitOp.num_qubits, entanglement="linear")
vqe = VQE(qubitOp, var_form, optimizer=optimizer)
# Set the quantum instance to be able to run only the last iteration
vqe.quantum_instance = quantum_instance

# Get energy from the stored jobs
energy = vqe._energy_evaluation(parameters=p0) + nuclear_repulsion_energy
print(energy)